# CNN Model

## Importing All the necessary Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
import warnings
warnings.filterwarnings("ignore")
import cv2
import pathlib

## Unarchieving 
Unzipping the achieved file downloaded from

`URL`- https://www.kaggle.com/datasets/masoudnickparvar/brain-tumor-mri-dataset

In [ ]:
import zipfile

zip_file_path = '/content/archive.zip'
destination_directory = '/content/archive'

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(destination_directory)

## Extracting
Extracting all the necessary pictures for model training and validation using Pathlib

In [ ]:
path=pathlib.Path.cwd()
path

PosixPath('/content')

In [ ]:
p=list(path.glob("archive"))
path=pathlib.Path(p[0])

In [ ]:
path

PosixPath('/content/archive')

In [ ]:
list(path.glob("Training/glioma/*.jpg"))[:5]

[PosixPath('/content/archive/Training/glioma/Tr-gl_0162.jpg'),
 PosixPath('/content/archive/Training/glioma/Tr-gl_0406.jpg'),
 PosixPath('/content/archive/Training/glioma/Tr-gl_0129.jpg'),
 PosixPath('/content/archive/Training/glioma/Tr-gl_0078.jpg'),
 PosixPath('/content/archive/Training/glioma/Tr-gl_0206.jpg')]

In [ ]:
X_dict={
    "glioma":list(path.glob("Training/glioma/*.jpg")),
    "meningioma":list(path.glob("Training/meningioma/*.jpg")),
    "notumor":list(path.glob("Training/notumor/*.jpg")),
    "pituitary":list(path.glob("Training/pituitary/*.jpg"))
}

In [ ]:
Y_dict={
    "glioma":0,
    "meningioma":1,
    "notumor":2,
    "pituitary":3
}

In [ ]:
j=plt.imread(list(X_dict.values())[1][0])
len(j.shape)

2

In [ ]:
x = []
y = []
k=[]
for names, image in X_dict.items():
    for img in image:
        a = plt.imread(str(img))
        if len(a.shape) == 2:
            a = cv2.cvtColor(a, cv2.COLOR_GRAY2RGB)
        if len(a.shape) == 3 and a.shape[2]==4:
            a = cv2.cvtColor(a, cv2.COLOR_RGBA2RGB)
        a = cv2.resize(a, (200, 200))

        if a.shape == (200, 200, 3):
            x.append(a)
            y.append(Y_dict[names])
        else:
            k.append(a)
x = np.array(x)
y = np.array(y)

In [ ]:
k

[]

In [ ]:
np.unique(y)

array([0, 1, 2, 3])

## Spliting The data

In [ ]:
from sklearn.model_selection import train_test_split
X,tx,Y,ty=train_test_split(x,y,test_size=0.2,random_state=42)

In [ ]:
np.unique(Y)

array([0, 1, 2, 3])

In [ ]:
np.unique(ty)

array([0, 1, 2, 3])

In [ ]:
X=X/255
tx=tx/255

# CNN Model

## Building the CNN Model
Defining the architecture of the CNN, including convolutional layers, activation functions, and pooling layers.

In [ ]:
da=keras.Sequential([
    keras.layers.RandomZoom(0.4,input_shape=(200,200,3)),
    keras.layers.RandomRotation(0.4),
])

In [ ]:
red=keras.callbacks.ReduceLROnPlateau(factor=0.8,monitor="val_accuracy",patience=3,verbose=1)

In [ ]:
early=keras.callbacks.EarlyStopping(monitor="val_accuracy",patience=30,restore_best_weights=True)

In [ ]:
model=keras.Sequential([
    da,
    keras.layers.Conv2D(filters=32,kernel_size=(3,3),padding="same",activation="relu"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling2D((2,2)),
    keras.layers.Conv2D(filters=64,kernel_size=(3,3),padding="same",activation="relu"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling2D((2,2)),
    keras.layers.Conv2D(filters=128,kernel_size=(3,3),padding="same",activation="relu"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling2D((2,2)),
    keras.layers.Conv2D(filters=256,kernel_size=(3,3),padding="same",activation="relu"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling2D((2,2)),
    keras.layers.Conv2D(filters=512,kernel_size=(3,3),padding="same",activation="relu"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling2D((2,2)),
    keras.layers.Flatten(),
    keras.layers.Dense(512,activation="relu",kernel_regularizer=keras.regularizers.L2(0.03)),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(256,activation="relu",kernel_regularizer=keras.regularizers.L2(0.03)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(128,activation="relu",kernel_regularizer=keras.regularizers.L2(0.03)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(64,activation="relu"),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(32,activation="relu"),
    keras.layers.Dense(4,activation="softmax"),
])

## Training the Model
Compiling and training the CNN model using an appropriate loss function and optimizer.

In [ ]:
model.compile(optimizer="adam",loss="sparse_categorical_crossentropy",metrics=["accuracy"])

In [ ]:
model.fit(X,Y,validation_data=(tx,ty),epochs=100,callbacks=[red,early])

Epoch 1/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 32s 146ms/step - accuracy: 0.4011 - loss: 29.8764 - val_accuracy: 0.2668 - val_loss: 8.5995 - learning_rate: 0.0010
Epoch 2/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 34s 127ms/step - accuracy: 0.5303 - loss: 7.1824 - val_accuracy: 0.2703 - val_loss: 5.6002 - learning_rate: 0.0010
Epoch 3/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 20s 125ms/step - accuracy: 0.6203 - loss: 4.3373 - val_accuracy: 0.3543 - val_loss: 4.1526 - learning_rate: 0.0010
Epoch 4/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 21s 126ms/step - accuracy: 0.6835 - loss: 2.8127 - val_accuracy: 0.6019 - val_loss: 2.2952 - learning_rate: 0.0010
Epoch 5/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 21s 129ms/step - accuracy: 0.6749 - loss: 2.0481 - val_accuracy: 0.6719 - val_loss: 1.6277 - learning_rate: 0.0010
Epoch 6/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 20s 125ms/step - accuracy: 0.7014 - loss: 1.5790 - val_accuracy: 0.6360 - val_loss: 1.4381 - learning_rate: 0.0010
Epoch 7/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 18s 128ms/step - accur

## Evaluating the Model
Assessing the model's performance using accuracy metric

In [ ]:
model.evaluate(tx,ty)

36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - accuracy: 0.9504 - loss: 0.2420


[0.2094089239835739, 0.9562554955482483]

# Saving the model
the model is saved for further testing 

In [ ]:
model.save("Brain_Tumour.keras")